# Assignment 3
## Javier Palomares
### Problem 1
Read Shannon’s 1948 paper ’A Mathematical Theory of Communication’. Focus on pages 1-19 (up
to Part II), the remaining part is more relevant for communication.
http://math.harvard.edu/~ctm/home/text/others/shannon/entropy/entropy.pdf
Summarize what you learned briefly (e.g. half a page).

In the paper, Shannon introduces entropy within the context of communication systems. The paper first introduces the channel used for transmitting information in the system. The channel is a system where a sequence of choices from a finite set $ \{s_1,s_2,...,s_n\}$ are transmitted from one point to another.  
The system has a capacity $ C=\lim_{T\to \inf} \frac{N(T)}{T}$ where $N(T)$ is the number of allowed signals of length $T$. If all sequences of $ \{s_1,s_2,...,s_n\}$ are allowed and the symbols have duration $t_1,t_2,...,t_n$ then it's shown that $C = logX_0$ where $X_0$ is the largest solution of $X^{-t_1} + X ^{-t_2} + ... + X ^{-t_n} =1$.  
This result is then taken to mean that the log of the number of possible signals increases linearly with time, and the capacity of a channel can be determined by the rate of increase. More importantly, this also means that statistical knowledge of the prodution of the sequence has the effect of reducing the required capacity of the channel.  
So we want to define a quantity which will measure how much inofrmation is produced by the source of the sequence, and at what rate this information is produced. This quantity, $H$, should have the following properties for a set of possible events each with probability $ p_1, p_2,...,p_n$:

1. $H$ should be continuous in the $p_i$
2. If all the $p_i$ are equal, then $H$ should be a monotonic function of n.
3. If a choice can be broken down into 2 successive choices, then the original $H$ should be the weighted sum of the individual H. $H(\frac{1}{2},\frac{1}{3},\frac{1}{6}) = H(\frac{1}{2},\frac{1}{2}) + \frac{1}{2} H (\frac{2}{3},\frac{1}{3})$ 
If turns out that the only $H$ satisfying the 3 properites is of the form  
$H = - K \sum_{i=1}^{n} p_i * log p_i, K>0$

This is the mathematical definition of what we call entropy for a discrete system.

### Problem 2: Scraping, Entropy and ICML papers.
ICML is a top research conference in Machine learning. Scrape all the pdfs of all ICML 2018 papers
from http://proceedings.mlr.press/v80/.
1. What are the top 10 common words in the ICML papers?

In [1]:
import heapq
import re
from urllib.request import urlopen
from urllib.parse import urlparse
import os
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfpage import PDFPage
from os import listdir
from os.path import isfile, join
# From PDFInterpreter import both PDFResourceManager and PDFPageInterpreter
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfdevice import PDFDevice
# Import this to raise exception whenever text extraction from PDF is not allowed
from pdfminer.pdfpage import PDFTextExtractionNotAllowed
from pdfminer.layout import LAParams, LTTextBox, LTTextLine
from pdfminer.converter import PDFPageAggregator
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string

URL = 'http://proceedings.mlr.press/v80/'
DOWNLOAD_PATH = "./pdfs/"
FREQUENCY_PATH = "./freqs/"
os.mkdir(DOWNLOAD_PATH)
os.mkdir(FREQUENCY_PATH)

def get_all_links():
    # connect to the url
    website = urlopen(URL)
    # get the html code
    html = website.read().decode('utf8')

    # use re.findall to get all the links
    links = re.findall('"((http|ftp)s?://.*?)"', html)
    return links

def get_pdf_links(links):
    pdfs = []
    pattern = re.compile(".*\.pdf")
    for tuple in links:
        link = tuple[0]
        matches = re.match(pattern,link)
        if matches:
            pdfs.append(link)
    return pdfs

def download_pdfs(pdfs):
    for pdf_link in pdfs:
        pdf = urlopen(pdf_link)
        o = urlparse(pdf_link)
        print("Downloading {}".format(pdf_link))
        file_name = os.path.basename(o[2])
        path = DOWNLOAD_PATH + file_name
        f = open(path,'wb+')
        f.write(pdf.read())
        f.close()


def read_pdfs():
    pdf_files = [f for f in listdir(DOWNLOAD_PATH) if isfile(join(DOWNLOAD_PATH, f))]
    word_freq = {}

    stopWords = set(stopwords.words('english'))
    stopWords.add('et')
    stopWords.add('al')
    for pdf_file in pdf_files:
        # word frequency in this one file. we'll write the
        # word frequency of per every single file in addition to
        # the word frequency across all files
        word_prob_this_file= {}
        file_len = 0
        print("reading {}".format(pdf_file))

        path = DOWNLOAD_PATH + pdf_file
        fp = open(path,'rb')
        parser = PDFParser(fp)
        document = PDFDocument(parser)
        extracted_text = ""

        if not document.is_extractable:
            raise PDFTextExtractionNotAllowed

        rsrcmgr = PDFResourceManager()

        # set parameters for analysis
        laparams = LAParams()

        # Create a PDFDevice object which translates interpreted information into desired format
        # Device needs to be connected to resource manager to store shared resources
        # device = PDFDevice(rsrcmgr)
        # Extract the decive to page aggregator to get LT object elements
        device = PDFPageAggregator(rsrcmgr, laparams=laparams)

        # Create interpreter object to process page content from PDFDocument
        # Interpreter needs to be connected to resource manager for shared resources and device
        interpreter = PDFPageInterpreter(rsrcmgr, device)

        # Ok now that we have everything to process a pdf document, lets process it page by page
        for page in PDFPage.create_pages(document):
            # As the interpreter processes the page stored in PDFDocument object
            interpreter.process_page(page)
            # The device renders the layout from interpreter
            layout = device.get_result()
            # Out of the many LT objects within layout, we are interested in LTTextBox and LTTextLine
            for lt_obj in layout:
                if isinstance(lt_obj, LTTextBox) or isinstance(lt_obj, LTTextLine):
                    text = lt_obj.get_text()
                    # remove any characters that pdfminer was not able to
                    # convert to unicode and converts to (cid:%%)
                    text = re.sub('(cid:[0-9]+)','',text)
                    # make all text lowercase
                    text = text.lower()
                    # remove line breaks
                    text = text.replace('-\n','')
                    # remove all punctuation from the text
                    text = text.translate(str.maketrans('', '', string.punctuation))
                    # remove new line
                    text = text.replace('\n',' ')
                    text = text.replace('−',' ')

                    extracted_text += text
        words = word_tokenize(extracted_text)

        for word in words:
            # don't include stop words or numeric words or single chars representing mathematical variables
            if (word not in stopWords) and (word.isnumeric()==False) and (len(word) > 1):
                # take all word to lowercase
                word = word.lower()
                # frequency across all files
                count = word_freq.get(word,0)
                word_freq[word] = count+1
                # frequency across this file
                count = word_prob_this_file.get(word,0)
                word_prob_this_file[word] = count+1
                file_len += 1
        fp.close()
        # write the word distribution of this file [0-1] values
        filename = FREQUENCY_PATH + pdf_file + ".txt"
        for word,count in word_prob_this_file.items():
            prob = 1.0 * count / (1.0 * file_len)
            word_prob_this_file[word] = prob
        write_freqs(word_prob_this_file,filename)

    return word_freq


def write_freqs(word_freq,filename):
    with open(filename, 'w+') as f:
        for word, count in word_freq.items():
            f.write('{} {}\n'.format(word,count))

def get_top_ten(word_freq):
    heap = [(-value, key) for key, value in word_freq.items()]
    largest = heapq.nsmallest(10, heap)
    largest = [(key, -value) for value, key in largest]
    return largest

In [11]:
links = get_all_links()
pdf_links = get_pdf_links(links)
download_pdfs(pdf_links)
word_freq = read_pdfs()
write_freqs(word_freq,'word_freqs.txt')
largest = get_top_ten(word_freq)
print(largest)

[('learning', 0.006927446954772004), ('algorithm', 0.004573534712676739), ('model', 0.004241958823744521), ('data', 0.003597552300027319), ('function', 0.003541009238337302), ('using', 0.0034807785856675014), ('set', 0.0034061048683268814), ('training', 0.002723900537066896), ('neural', 0.0026630552859004653), ('gradient', 0.0026572166001824745)]


Top ten words are, in order:
1. learning
2. algorithm
3. model
4. data
5. function
6. using
7. set
8. training
9. neural
10. gradient

Note: I threw out english stopwords as well as single character words I assumed represented mathematical variables, and any symbols that pdfminer was unable to convert to unicode

2. Let Z be a randomly selected word in a randomly selected ICML paper. Estimate the entropy
of Z.

In [18]:
import math
import numpy as np
from alphabet_detector import AlphabetDetector
from os import listdir
from os.path import isfile, join

WORD_FREQ_FILE= 'word_freqs.txt'
PARAGRAPH_NUM_WORDS = 30
FREQUENCY_PATH = "./freqs/"

K = 1.0
def get_word_prob(all_words=False):
    ad = AlphabetDetector()
    word_probs = {}
    total_word_count = 0
    with open(WORD_FREQ_FILE,'r') as f:
        line = f.readline()
        while line:
            tokens = line.split()
            word = tokens[0]
            # Only consider latin words by default
            if all_words or ad.only_alphabet_chars(word,"LATIN"):
                count = int(tokens[1])
                word_probs[word] = count
                total_word_count += count
            line = f.readline()
    # convert the count to a prob by dividing by the total count
    for word,count in word_probs.items():
        word_probs[word] = count / total_word_count
    return word_probs

# return a list of the frequency a word per file
def get_all_freq_files():
    files = [f for f in listdir(FREQUENCY_PATH) if isfile(join(FREQUENCY_PATH, f))]

    word_prob_all_files = []
    for file in files:
        file_word_prob = {}
        path = FREQUENCY_PATH + file
        with open(path,'r') as f:
            line = f.readline()
            while line:
                tokens = line.split()
                word = tokens[0]
                prob = float(tokens[1])
                file_word_prob[word] = prob
                line = f.readline()
        word_prob_all_files.append(file_word_prob)
    return word_prob_all_files



def compute_entropy_random_word_random_paper(words,word_prob_all_files):
    H = 0
    # iterate over all the words
    num_words = len(words)
    num_files = len(word_prob_all_files)
    for word in words:
        p = 0
        # iterate over the word distribution per file
        for word_prob in word_prob_all_files:
            # all files are equally likely, so divide by the number of files
            prob = word_prob.get(word,0) * 1.0 / num_files
            p += prob
        h = -1.0 * K * p * math.log(p,2)
        H += h
    return H

def verify_word_probs(word_probs):
    # verify the word probs sum up to 1.0
    s = 0.0
    for word,p in word_probs.items():
        s += p
    print("total prob: {}".format(s))

def first_order_synthesized_paragraph(word_probs,par_len):
    # synthesize a paragraph of the given length with the
    # given word probabilities
    words = list(word_probs.keys())
    weights = list(word_probs.values())
    paragraph = np.random.choice(words,par_len,p=weights)
    return ' '.join(paragraph)


In [ ]:
all_word_probs = get_word_prob(True)
words = all_word_probs.keys()
word_prob_all_files = get_all_freq_files()
H = compute_entropy_random_word_random_paper(words,word_prob_all_files)
print("The computed entropy for the pdf text is {}".format(H))

3. Synthesize a random paragraph using the marginal distribution over words.